In [37]:
import os
import re
import numpy as np
import matplotlib.pyplot as plt

path='DataSG056'
file_path=[]
for filename in os.listdir(path):
    file_path.append((os.path.join(path,filename)))


### read data
* first 6 columns are dates `2021 04 01 00 00 00`
* 7th column -> G1-F60 in volt 
* 8th column -> G2-F60 in volt
* 9th column -> Br1-F60
* 10th column -> Br2-F60

In [39]:
data=np.array([])
for address in file_path:
    if data.size==0:
        data=np.loadtxt(address,skiprows=43,unpack=True)
    else:
        nd=np.loadtxt(address,skiprows=43,unpack=True)
        data=np.column_stack((data,nd))

In [3]:
rege1=r"(?<=\[GRAVITY\_CAL\_1\_UGAL\_V\]\s{10}).*$"
rege2=r"[GRAVITY_CAL_2_UGAL_V]"
str='[GRAVITY_CAL_1_UGAL_V]          -40.1100'

print(re.findall(rege1,str))

file1=open(file_path[0])
l=file1.readlines( )
grav1=[]

for x in l:
    new_x=x.splitlines()
    grav1=re.findall(rege1,new_x[0])
print(grav1)

['-40.1100']
[]
